In [1]:
# -*- coding: utf-8 -*-
"""
# 🤖 From LLMs to Operating Systems: The MemGPT Approach

Inspired by the [MemGPT paper](https://arxiv.org/pdf/2310.08560) and the [DeepLearning.AI course](https://learn.deeplearning.ai/courses/llms-as-operating-systems-agent-memory), this notebook explores how to overcome the memory limitations of LLMs by treating them like the CPU of a traditional operating system.
"""

#@title ## 1. Setup and Configuration
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import json
# Import the function from your helper file
from helper import get_gemini_api_key

# --- Configuration ---
# Note: To run this, you'll need a helper.py file with a get_gemini_key() function.
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key not found in helper.py")
    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully!")
except Exception as e:
    print(f"🚨 Error configuring API key: {e}")
    print("Please ensure your helper.py file is in the same directory and returns a valid API key.")

"""---
## Concept 1: The Memory Hierarchy 🧠

Just like a computer has fast RAM and slow disk storage, an agent has a tiered memory system. The key is to intelligently manage what information is loaded into the LLM's limited "working memory" (the context window).
"""

#@title ### Example: Main Context (RAM) vs. External Context (Disk)

# The LLM's direct context window.
# Fast, but limited in size.
main_context = {
    "system_prompt": "You are a helpful assistant.",
    "working_memory": {"user_name": "Alex"},
    "chat_history": ["User: Hi!", "Agent: Hello Alex!"]
}

# A vector database or file system where long-term knowledge is stored.
# Vast, but slower to access.
external_context = {
    "document_id_123": "The company's vacation policy allows for 20 days off per year...",
    "document_id_456": "The Q3 financial report shows a 10% increase in revenue..."
}

print("--- Main Context (RAM) ---")
print(json.dumps(main_context, indent=2))
print("\n--- External Context (Disk) ---")
print(json.dumps(external_context, indent=2))

"""---
## Concept 2: Self-Editing Memory ✍️

The core idea of MemGPT: give the LLM the tools to manage its own memory. Instead of us deciding what's important, the agent learns to save critical information itself.
"""

#@title ### Example: Saving User Information with a Tool

# 1. We start with an empty memory block for the 'human'.
agent_memory = {"human": ""}

# 2. Define the tool implementation in Python.
def core_memory_save(section: str, memory: str):
    """A simple function to update our agent's memory dictionary."""
    if section in agent_memory:
        agent_memory[section] += f"\n{memory}"
        print(f"✅ Memory updated in section: '{section}'")
    else:
        print(f"❌ Error: Section '{section}' not found in memory.")

# 3. Simulate the LLM's decision to call the tool.
# The user says "My name is Bob." The LLM decides this is important.
llm_tool_call = {
    "name": "core_memory_save",
    "arguments": {
        "section": "human",
        "memory": "The human's name is Bob."
    }
}

print("Initial memory:", agent_memory)

# 4. Our code executes the tool call.
core_memory_save(**llm_tool_call["arguments"])

print("Final memory:", agent_memory)

"""---
## Concept 3: Agentic RAG 📚

This memory system enables a powerful, agent-driven RAG process. The agent actively decides when it needs more information and goes to get it from its external memory.
"""

#@title ### Example: Answering a Question with RAG

# 1. The user asks a question the agent can't answer from its working memory.
user_question = "What is the company's vacation policy?"

# 2. Simulate the LLM's decision to search external memory.
llm_tool_call_rag = {
    "name": "archival_memory_search",
    "arguments": {
        "query": "vacation policies"
    }
}

# 3. Define the RAG tool implementation.
def archival_memory_search(query: str):
    """Simulates searching our external context (vector DB)."""
    print(f"🔎 Searching external memory for: '{query}'")
    if "vacation" in query:
        return external_context["document_id_123"]
    return "No relevant information found."

# 4. Our code executes the tool call.
retrieved_context = archival_memory_search(**llm_tool_call_rag["arguments"])
print(f"📄 Retrieved Context: '{retrieved_context}'")

# 5. The retrieved context is "augmented" into the next prompt,
# allowing the LLM to generate a fact-based answer.
final_prompt = f"""
Context:
{retrieved_context}

Question:
{user_question}

Answer the question based on the provided context.
"""

print("\n--- Final Prompt for Generation ---")
print(final_prompt)

"""---
## Concept 4: Memory Redaction in Multi-Agent Systems 🔒

When multiple agents share memory, we need a way to control access to sensitive information. An "Admin" agent with special privileges can redact confidential data before sharing it with other agents.
"""

#@title ### Example: Redacting PII for a Junior Agent

# 1. A shared document with sensitive information.
shared_document = {
    "Name": "John Doe",
    "Phone": "555-1234",
    "Address": "123 Main St",
    "Skills": "Python, SQL"
}

# 2. The "Admin Controller" has a redaction tool.
def redact_memory(document, fields_to_redact):
    """Redacts specified fields from a document."""
    redacted_doc = document.copy()
    for field in fields_to_redact:
        if field in redacted_doc:
            redacted_doc[field] = "[REDACTED]"
    return redacted_doc

# 3. A Junior Analyst agent requests the document. The Admin Controller intercepts.
fields_to_redact = ["Name", "Phone", "Address"]
redacted_document = redact_memory(shared_document, fields_to_redact)

print("--- Original Document (Visible to Hiring Manager) ---")
print(json.dumps(shared_document, indent=2))

print("\n--- Redacted Document (Visible to Junior Analyst) ---")
print(json.dumps(redacted_document, indent=2))


✅ Gemini API configured successfully!
--- Main Context (RAM) ---
{
  "system_prompt": "You are a helpful assistant.",
  "working_memory": {
    "user_name": "Alex"
  },
  "chat_history": [
    "User: Hi!",
    "Agent: Hello Alex!"
  ]
}

--- External Context (Disk) ---
{
  "document_id_123": "The company's vacation policy allows for 20 days off per year...",
  "document_id_456": "The Q3 financial report shows a 10% increase in revenue..."
}
Initial memory: {'human': ''}
✅ Memory updated in section: 'human'
Final memory: {'human': "\nThe human's name is Bob."}
🔎 Searching external memory for: 'vacation policies'
📄 Retrieved Context: 'The company's vacation policy allows for 20 days off per year...'

--- Final Prompt for Generation ---

Context:
The company's vacation policy allows for 20 days off per year...

Question:
What is the company's vacation policy?

Answer the question based on the provided context.

--- Original Document (Visible to Hiring Manager) ---
{
  "Name": "John Doe",


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
